# Set / cycle aesthetic models implemented with just NumPy

This notebook serves as a reference for using the machine-learning-derived aesthetic-preference models in plain NumPy. It eliminates the dependency on TensorFlow, allows for much quicker startup time, and eliminates the need to construct the model differently for different set / cycle sizes.

In [1]:
import gzip
import numpy as np
import colorspacious

In [2]:
def to_jab(color):
    """
    Convert hex color code (without `#`) to CAM02-UCS.
    """
    rgb = [(int(i[:2], 16), int(i[2:4], 16), int(i[4:], 16)) for i in color]
    jab = [colorspacious.cspace_convert(i, "sRGB255", "CAM02-UCS") for i in rgb]
    return np.array(jab, dtype=np.float32)


def sort_colors_by_j(colors):
    """
    Sorts colors by CAM02-UCS J' axis.
    """
    return colors[np.lexsort(colors[:, ::-1].T, 0)]


def sort_colors_by_a(colors):
    """
    Sorts colors by CAM02-UCS a' axis.
    """
    return colors[np.argsort(colors[:, ::-1].T[1])]


def sort_colors_by_b(colors):
    """
    Sorts colors by CAM02-UCS b' axis.
    """
    return colors[np.argsort(colors[:, ::-1].T[2])]

In [3]:
# These four functions are based on functions in: https://github.com/keras-team/keras/blob/2.3.0/keras/backend/numpy_backend.py


def conv(x, w):
    y = []
    for j in range(w.shape[1]):
        _y = [np.convolve(x[k], w[k, j], "same") for k in range(w.shape[0])]
        y.append(np.sum(np.stack(_y, axis=-1), axis=-1))
    return np.array(y)


def depthwise_conv(x, w):
    y = []
    for j in range(w.shape[0]):
        _y = [np.convolve(x[j], w[j, k], "same") for k in range(w.shape[1])]
        y.append(np.stack(_y, axis=0))
    return np.concatenate(y, axis=0)


def elu(x):
    return x * (x > 0) + (np.exp(x) - 1) * (x < 0)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
class Dense(object):
    def __init__(self, kernel, bias):
        self.kernel = kernel
        self.bias = bias

    def __call__(self, inputs):
        outputs = np.dot(inputs, self.kernel)
        outputs += self.bias
        return elu(outputs)


class SeparableConv1D(object):
    def __init__(self, depthwise_kernel, pointwise_kernel, bias):
        self.depthwise_kernel = depthwise_kernel
        self.pointwise_kernel = pointwise_kernel
        self.bias = bias

    def __call__(self, inputs):
        outputs = depthwise_conv(inputs, self.depthwise_kernel)
        outputs = conv(outputs, self.pointwise_kernel)
        outputs += self.bias
        return elu(outputs)

In [5]:
class SetModel(object):
    def __init__(self, filename):
        # Load model weights
        layers = []
        with gzip.open(filename, "rb") as infile:
            weight_file = np.load(infile)
            for i in range(weight_file["ensemble_count"]):
                layers.append({})
                for key in ["1j", "2j", "1a", "2a", "1b", "2b"]:
                    kernel = weight_file[key + f"_{i:03d}_kernel"]
                    bias = weight_file[key + f"_{i:03d}_bias"]
                    layers[i][key] = Dense(kernel, bias)
                for key in ["3j", "4j", "5j", "3a", "4a", "5a", "3b", "4b", "5b"]:
                    depthwise_kernel = weight_file[key + f"_{i:03d}_depthwise_kernel"]
                    pointwise_kernel = weight_file[key + f"_{i:03d}_pointwise_kernel"]
                    bias = weight_file[key + f"_{i:03d}_bias"]
                    layers[i][key] = SeparableConv1D(
                        depthwise_kernel, pointwise_kernel, bias
                    )
        self.all_layers = layers

    @staticmethod
    def _eval_ensemble_instance(layers, input_a):
        """
        layers: dict with callable layers
        input_a: [colors sorted by J', colors sorted by a', colors sorted by b']; shape=(3 * num_colors,)
        """
        num_colors = input_a[0].shape[0] // 3

        # Create network
        inputs_a_j = [input_a[0][i * 3 : (i + 1) * 3] for i in range(num_colors)]
        inputs_a_a = [input_a[1][i * 3 : (i + 1) * 3] for i in range(num_colors)]
        inputs_a_b = [input_a[2][i * 3 : (i + 1) * 3] for i in range(num_colors)]

        # Share layers between colors
        x_a_j = [layers["1j"](i / 100) for i in inputs_a_j]
        x_a_a = [layers["1a"](i / 100) for i in inputs_a_a]
        x_a_b = [layers["1b"](i / 100) for i in inputs_a_b]

        x_a_j = [layers["2j"](i) for i in x_a_j]
        x_a_a = [layers["2a"](i) for i in x_a_a]
        x_a_b = [layers["2b"](i) for i in x_a_b]

        # Combine colors into sets
        x_a_j = np.vstack(x_a_j).T
        x_a_a = np.vstack(x_a_a).T
        x_a_b = np.vstack(x_a_b).T

        # Share layers between color sets
        x_a_j = layers["3j"](x_a_j)
        x_a_a = layers["3a"](x_a_a)
        x_a_b = layers["3b"](x_a_b)

        x_a_j = layers["4j"](x_a_j)
        x_a_a = layers["4a"](x_a_a)
        x_a_b = layers["4b"](x_a_b)

        x_a_j = layers["5j"](x_a_j)
        x_a_a = layers["5a"](x_a_a)
        x_a_b = layers["5b"](x_a_b)

        # Average outputs
        x_a_j = np.mean(x_a_j)
        x_a_a = np.mean(x_a_a)
        x_a_b = np.mean(x_a_b)

        # Final non-linear activation
        x_a_j = sigmoid(x_a_j)
        x_a_a = sigmoid(x_a_a)
        x_a_b = sigmoid(x_a_b)

        # Final averaging of sub-ensemble
        return np.mean([x_a_j, x_a_a, x_a_b])

    def __call__(self, rgb_colors, average=True):
        jab = to_jab(rgb_colors)
        sorted_by_j = sort_colors_by_j(jab).flatten()
        sorted_by_a = sort_colors_by_a(jab).flatten()
        sorted_by_b = sort_colors_by_b(jab).flatten()
        inputs = (sorted_by_j, sorted_by_a, sorted_by_b)
        scores = np.array(
            [SetModel._eval_ensemble_instance(l, inputs) for l in self.all_layers]
        )
        if average:
            return np.mean(scores)
        return scores

In [6]:
class CycleModel(object):
    def __init__(self, filename):
        # Load model weights
        layers = []
        with gzip.open(filename, "rb") as infile:
            weight_file = np.load(infile)
            for i in range(weight_file["ensemble_count"]):
                layers.append({})
                for key in ["1", "2"]:
                    kernel = weight_file[key + f"_{i:03d}_kernel"]
                    bias = weight_file[key + f"_{i:03d}_bias"]
                    layers[i][key] = Dense(kernel, bias)
                for key in ["3", "4", "5"]:
                    depthwise_kernel = weight_file[key + f"_{i:03d}_depthwise_kernel"]
                    pointwise_kernel = weight_file[key + f"_{i:03d}_pointwise_kernel"]
                    bias = weight_file[key + f"_{i:03d}_bias"]
                    layers[i][key] = SeparableConv1D(
                        depthwise_kernel, pointwise_kernel, bias
                    )
        self.all_layers = layers

    @staticmethod
    def _eval_ensemble_instance(layers, input_a):
        """
        layers: dict with callable layers
        input_a: color cycle; shape=(3 * num_colors,)
        """
        num_colors = input_a.shape[0] // 3

        # Create network
        inputs_a = [input_a[i * 3 : (i + 1) * 3] for i in range(num_colors)]

        # Share layers between colors
        x_a = [layers["1"](i / 100) for i in inputs_a]
        x_a = [layers["2"](i) for i in x_a]

        # Combine colors into sets
        x_a = np.vstack(x_a).T

        # Share layers between color sets
        x_a = layers["3"](x_a)
        x_a = layers["4"](x_a)
        x_a = layers["5"](x_a)

        # Average outputs
        x_a = np.mean(x_a)

        # Final non-linear activation
        return sigmoid(x_a)

    def __call__(self, rgb_colors, average=True):
        jab = to_jab(rgb_colors).flatten()
        scores = np.array(
            [CycleModel._eval_ensemble_instance(l, jab) for l in self.all_layers]
        )
        if average:
            return np.mean(scores)
        return scores

In [7]:
set_model = SetModel("set_model_weights.npz.gz")

In [8]:
set_model(["5790fc", "f89c20", "e42536", "964a8b", "9c9ca1", "7a21dd"], False)

array([0.9407356 , 0.92325067, 0.91179653, 0.90746855, 0.97235842,
       0.87814595, 0.95064555, 0.95652208, 0.95896659, 0.93452655,
       0.95849617, 0.95634684, 0.92588308, 0.89528361, 0.92886266,
       0.93745349, 0.91188076, 0.92659491, 0.9146983 , 0.95082445,
       0.93291935, 0.90114288, 0.93595538, 0.95127765, 0.95795301,
       0.93861053, 0.96140436, 0.95401919, 0.92817434, 0.88656765,
       0.89589191, 0.97675997, 0.94061453, 0.91865082, 0.95474508,
       0.94461286, 0.92363482, 0.96499798, 0.92938569, 0.96354053,
       0.94115221, 0.96538744, 0.92923351, 0.9368276 , 0.94854879,
       0.95069497, 0.94660813, 0.94894713, 0.9239899 , 0.91478789,
       0.95417172, 0.97155175, 0.87584965, 0.9221434 , 0.93746551,
       0.97827544, 0.9734127 , 0.98348642, 0.92747586, 0.93725532,
       0.98018789, 0.91959197, 0.93972237, 0.96397076, 0.94397894,
       0.95660088, 0.92827893, 0.94917383, 0.91458699, 0.97417309,
       0.95993975, 0.96077677, 0.95813232, 0.93259952, 0.95964

In [9]:
set_model(["5790fc", "f89c20", "e42536", "964a8b", "9c9ca1", "7a21dd"])

0.9384832896546818

In [10]:
cycle_model = CycleModel("cycle_model_weights.npz.gz")

In [11]:
cycle_model(["5790fc", "f89c20", "e42536", "964a8b", "9c9ca1", "7a21dd"], False)

array([0.77880305, 0.74438472, 0.81492511, 0.70274252, 0.78859189,
       0.70707072, 0.80014955, 0.82867692, 0.87105315, 0.91213424,
       0.87378331, 0.74812133, 0.85634481, 0.6541449 , 0.83843975,
       0.89770209, 0.84243689, 0.85762795, 0.86561825, 0.60064065,
       0.80427225, 0.89478529, 0.80474089, 0.92233487, 0.91443815,
       0.8102286 , 0.63196214, 0.83966476, 0.82920736, 0.90919499,
       0.8448862 , 0.92216871, 0.8682478 , 0.5990899 , 0.84469815,
       0.75213969, 0.79534706, 0.64202322, 0.85766183, 0.90605724,
       0.69709626, 0.92023759, 0.80421394, 0.76639233, 0.86526482,
       0.7743707 , 0.80446451, 0.71651572, 0.85954928, 0.78405863,
       0.86404835, 0.7947659 , 0.81332994, 0.69471278, 0.77248851,
       0.85317296, 0.53549137, 0.91477984, 0.68973683, 0.85540784,
       0.7795836 , 0.66692666, 0.74429577, 0.84926695, 0.54264075,
       0.65804364, 0.79430409, 0.70697147, 0.94339815, 0.86959849,
       0.91572258, 0.7845324 , 0.87570527, 0.90863009, 0.89174

In [12]:
cycle_model(["5790fc", "f89c20", "e42536", "964a8b", "9c9ca1", "7a21dd"])

0.793204612141789